## Overview
    This file contains 3 functions: findDistrict(), findDistrictDataframe(), and findCitipyDistrict()
    
### findDistrict( lat, lng, googleApiKey )
    Dependencies: json, requests
    Inputs: lat(int), lng(int), googleApiKey(str)
    Output: locality(str) if successful, "null"(str) if unsuccessful
    
    Description:
    findDistrict will use the provided coordinates to build a url and retrive data from that endpoint. It'll
    then search through the address components till it finds the locality, and then return the the name of 
    the locality. If it is unable to find the locality component, it will return "null". If it gets to the end
    of the function without returning the address, it will also return "null"
    
### findDistrictDataframe( df, googleApiKey )
    Dependencies: json, request, pandas, findDistrict()
    Inputs: df(dataframe), googleApiKey(str)
    Output: dataframe
    
    Description: findDistrictDataframe applies the functionality of findDistrict to a dataframe. It'll take two 
    columns from the dataframe (NOTE: COLUMNS MUST BE CALLED latitude AND longitude) and convert them to a list.
    Using the elements from that list, it'll call findDistrict, append the value to the list, and later add that
    list to a new column in the dataframe called "district"
    
### findCitipyDistrict( df )
    Dependencies: citipy, pandas
    Inputs: df(dataframe)
    Output: dataframe
    
    Description: Operates in the same manner as findDistrictDataframe, except instead of calling findDistrict, it
    uses the citipy library to return a result. Often provides a larger locality than findDistrict.

In [44]:
import json
import requests
import pandas as pd
from citipy import citipy
from pprint import pprint

def findDistrict(lat, lng, googleApiKey): #Outputs a district name in the form of a string.
    endpointURL = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={googleApiKey}"
    try:
        response = requests.get(endpointURL).json()
        addressComponents = response["results"][0]['address_components']
        for i in range(len(addressComponents)):
            if (addressComponents[i]['types'][0] == 'neighborhood'):
                return addressComponents[i]['long_name']
    except (KeyError, ValueError):
        print("KeyError or ValueError occured, district name was not retrivable.")
    return "null"

def districtDataframe(df, googleApiKey):
    latitude = df['latitude'].tolist()
    longitude = df['longitude'].tolist()
    district = []
    for i in range(len(latitude)):
        district.append(findDistrict(latitude[i], longitude[i], googleApiKey))
    df['district'] = district
    pprint(df)
    return df

In [40]:
def citipyDistrict(df):
    latitude = df['latitude'].tolist()
    longitude = df['longitude'].tolist()
    district = []
    for i in range(len(latitude)):
        city = citipy.nearest_city(latitude[i],longitude[i])
        district.append(city.city_name)
    df['district'] = district
    pprint(df)
    return df